In [7]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
from importlib import reload
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
from functools import reduce
import open3d as o3d
from multiprocessing.pool import ThreadPool


In [8]:
#ifc_file = ifcopenshell.open(r"IFC Files\Project1.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Duplex.ifc")
ifc_file = ifcopenshell.open(r"IFC Files\Clinic.ifc")

def open3d_block_by_element(ifc_file): 
    start_time = time.time()
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    exclude_list = ["IfcOpening", "IfcSpace"]
    building_element_blocks = {}
    all_meshes = []

    if iterator.initialize():
        while True:
            shape = iterator.get()
            if shape.type not in exclude_list:
                product = ifc_file.by_guid(shape.guid)
                building_element_blocks[product] = []
              
                faces = np.array(shape.geometry.faces)             
                verts = np.array(shape.geometry.verts).reshape(-1, 3)
                mesh = o3d.geometry.TriangleMesh()
                mesh.vertices = o3d.utility.Vector3dVector(verts)
                mesh.triangles = o3d.utility.Vector3iVector(faces.reshape(-1, 3))
                
                all_meshes.append(mesh)

            if not iterator.next():
                break
    end_time = time.time()
    print(f"Time taken to convert objects: {end_time - start_time:.4f} seconds")

    return all_meshes

def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def open3d_to_pyvista(point_cloud_o3d):
    """
    Convert an Open3D point cloud to a PyVista point cloud.

    Parameters:
    - point_cloud_o3d: The Open3D point cloud.

    Returns:
    - A PyVista `PolyData` object.
    """
    # Extract points from Open3D point cloud
    points = np.asarray(point_cloud_o3d.points)

    # Create a PyVista PolyData object
    point_cloud_pv = pv.PolyData(points)

    return point_cloud_pv

def voxelize_space(bounds, pcd_list, voxel_size):
    """Create a 3d grid and check the intersections of the meshes with the grid."""
    start_time = time.time()
    grid = create_uniform_grid(bounds, voxel_size)
    empty_grid = create_uniform_grid(bounds, voxel_size)
    
    # create an empty 3d array in the same dimensions of the grid
    dims = grid.dimensions
    grid_values = np.zeros((dims[0]-1, dims[1]-1, dims[2]-1))

    grid_bounds = grid.bounds

    for point in pcd_list.points:
        j = int ((point[0] - grid_bounds[0])/voxel_size)
        i = int ((point[1] - grid_bounds[2])/voxel_size)
        k = int ((point[2] - grid_bounds[4])/voxel_size)

        # Set the value of the cell to 1
        if i < dims[0]-1 and j < dims[1]-1 and k < dims[2]-1:
            grid_values[i,j,k] = 1

    #assign empty array to the grid
    grid.cell_data['data'] = grid_values.flatten(order='F').astype(bool)
    end_time = time.time()
    print(f"Time taken to rasterize: {end_time - start_time:.4f} seconds")
    return grid, empty_grid

def get_sampling_points(mesh, points_per_unit_area=120):
    
    # Calculate the surface area of the mesh
    area = mesh.get_surface_area()

    # Calculate the number of points to sample
    N = int(area * points_per_unit_area)

    return N

def create_point_cloud_for_mesh(mesh):
    N = get_sampling_points(mesh)
    pcd = mesh.sample_points_uniformly(N)
    pcd_pv = open3d_to_pyvista(pcd)
    return pcd_pv

def create_point_cloud(all_meshes):
    with ThreadPool(processes=multiprocessing.cpu_count()) as pool:
        point_cloud_list = pool.map(create_point_cloud_for_mesh, all_meshes)
    return point_cloud_list

In [9]:
### CONVERSION FROM IFC TO MESH ###
all_meshes = open3d_block_by_element(ifc_file)

print(f'Combining meshes...')
# combine all meshes into one mesh and find the bounds
combined_mesh = reduce(lambda m1, m2: m1 + m2, all_meshes)
bbox = combined_mesh.get_axis_aligned_bounding_box()
xmin, ymin, zmin = bbox.get_min_bound()
xmax, ymax, zmax = bbox.get_max_bound()
bounds = np.array([xmin, xmax, ymin, ymax, zmin, zmax])


### POINT CLOUD CREATION ###
start_time = time.time()

point_cloud_list = create_point_cloud(all_meshes)

#merge all the list of point clouds into one
pcd_pv = pv.PolyData()
pcd_pv.points = np.vstack(list(map(lambda x: x.points, point_cloud_list)))
    
end_time = time.time()
print(f"Time taken to create point cloud: {end_time - start_time:.4f} seconds")


### RASTERIZATION ###
voxel_size = 0.5
grid, empty_grid = voxelize_space(bounds, pcd_pv, voxel_size)

### VISUALIZATION ###
p = pv.Plotter()
p.add_mesh(pcd_pv, color="blue", point_size=20.0, render_points_as_spheres=True)
p.add_mesh(empty_grid, opacity=0.3,show_edges=True)
p.add_mesh(grid.extract_cells(grid.cell_data['data']), color="red", opacity=0.6,show_edges=True)
p.show()

Time taken to convert objects: 22.0776 seconds
Combining meshes...
Time taken to create point cloud: 2.5362 seconds
Time taken to rasterize: 26.6966 seconds


Widget(value="<iframe src='http://localhost:52143/index.html?ui=P_0x1f7ed653be0_2&reconnect=auto' style='width…